In [23]:
! rm -rf nano-BERT
! git clone https://github.com/StepanTita/nano-BERT.git

Cloning into 'nano-BERT'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 13 (delta 2), reused 13 (delta 2), pack-reused 0
Receiving objects: 100% (13/13), 2.08 MiB | 4.27 MiB/s, done.
Resolving deltas: 100% (2/2), done.


In [1]:
import sys

sys.path.append('/content/nano-BERT')

In [2]:
import json
from collections import Counter

import torch
import torch.nn.functional as F
from sklearn.metrics import accuracy_score, f1_score
from tqdm import tqdm

from nano_bert.model import NanoBertForClassification
from nano_bert.tokenizer import WordTokenizer

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
data = None
with open('nano-BERT/data/dataset.json') as f:
    data = json.loads(f.read())

In [5]:
vocab = set()
for id in data:
    vocab |= set(data[id]['post_tokens'])

In [6]:
ids_split = None
with open('nano-BERT/data/post_id_divisions.json') as f:
    ids_split = json.loads(f.read())

In [7]:
def encode_label(label):
    if label == 'hatespeech':
        return 0
    elif label == 'normal':
        return 1
    elif label == 'offensive':
        return 0  # 2
    raise Exception(f'Unknown Label: {label}!')


class HateDataloader:
    def __init__(self, data, ids_split, tokenizer, label_encoder, batch_size):
        def get_label(annotators):
            return Counter([anno['label'] for anno in annotators]).most_common(1)[0][0]

        self.splits = {
            'train': [data[post_id]['post_tokens'] for post_id in tqdm(data, desc='Train') if
                      post_id in ids_split['train']],
            'test': [data[post_id]['post_tokens'] for post_id in tqdm(data, desc='Test') if
                     post_id in ids_split['test']],
            'val': [data[post_id]['post_tokens'] for post_id in tqdm(data, desc='Val') if post_id in ids_split['val']]
        }

        self.labels = {
            'train': [get_label(data[post_id]['annotators']) for post_id in tqdm(data, desc='Train Labels') if
                      post_id in ids_split['train']],
            'test': [get_label(data[post_id]['annotators']) for post_id in tqdm(data, desc='Test Labels') if
                     post_id in ids_split['test']],
            'val': [get_label(data[post_id]['annotators']) for post_id in tqdm(data, desc='Val Labels') if
                    post_id in ids_split['val']]
        }

        self.tokenized = {
            'train': [tokenizer(record).unsqueeze(0) for record in
                      tqdm(self.splits['train'], desc='Train Tokenization')],
            'test': [tokenizer(record).unsqueeze(0) for record in tqdm(self.splits['test'], desc='Test Tokenization')],
            'val': [tokenizer(record).unsqueeze(0) for record in tqdm(self.splits['val'], desc='Val Tokenization')],
        }

        self.encoded_labels = {
            'train': [label_encoder(label) for label in tqdm(self.labels['train'], desc='Train Label Encoding')],
            'test': [label_encoder(label) for label in tqdm(self.labels['test'], desc='Test Label Encoding')],
            'val': [label_encoder(label) for label in tqdm(self.labels['val'], desc='Val Label Encoding')],
        }

        self.curr_batch = 0
        self.batch_size = batch_size
        self.iterate_split = None

    def peek(self, split):
        return {
            'input_ids': self.splits[split][self.batch_size * self.curr_batch:self.batch_size * (self.curr_batch + 1)],
            'label_ids': self.labels[split][self.batch_size * self.curr_batch:self.batch_size * (self.curr_batch + 1)],
        }

    def take(self, split):
        batch = self.splits[split][self.batch_size * self.curr_batch:self.batch_size * (self.curr_batch + 1)]
        labels = self.labels[split][self.batch_size * self.curr_batch:self.batch_size * (self.curr_batch + 1)]
        self.curr_batch += 1
        return {
            'input_ids': batch,
            'label_ids': labels,
        }

    def peek_tokenized(self, split):
        return {
            'input_ids': torch.cat(
                self.tokenized[split][self.batch_size * self.curr_batch:self.batch_size * (self.curr_batch + 1)],
                dim=0),
            'label_ids': torch.tensor(
                self.encoded_labels[split][self.batch_size * self.curr_batch:self.batch_size * (self.curr_batch + 1)],
                dtype=torch.long),
        }

    def take_tokenized(self, split):
        batch = self.tokenized[split][self.batch_size * self.curr_batch:self.batch_size * (self.curr_batch + 1)]
        labels = self.encoded_labels[split][self.batch_size * self.curr_batch:self.batch_size * (self.curr_batch + 1)]
        self.curr_batch += 1
        return {
            'input_ids': torch.cat(batch, dim=0),
            'label_ids': torch.tensor(labels, dtype=torch.long),
        }

    def get_split(self, split):
        self.iterate_split = split
        return self

    def steps(self, split):
        return len(self.tokenized[split]) // self.batch_size

    def __iter__(self):
        self.reset()
        return self

    def __next__(self):
        if self.batch_size * self.curr_batch < len(self.splits[self.iterate_split]):
            return self.take_tokenized(self.iterate_split)
        else:
            raise StopIteration

    def reset(self):
        self.curr_batch = 0

In [41]:
NUM_EPOCHS = 100
BATCH_SIZE = 32
MAX_SEQ_LEN = 128
LEARNING_RATE = 1e-2

In [34]:
tokenizer = WordTokenizer(vocab=vocab, max_seq_len=MAX_SEQ_LEN)
tokenizer

Tokenizer[vocab=28842,self.special_tokens=['[PAD]', '[CLS]', '[SEP]', '[UNK]'],self.sep=' ',self.max_seq_len=128]

In [35]:
dataloader = HateDataloader(data, ids_split, tokenizer, encode_label, batch_size=BATCH_SIZE)

Val Label Encoding: 100%|██████████| 1922/1922 [00:00<00:00, 1065624.89it/s]


In [36]:
bert = NanoBertForClassification(
    vocab_size=len(tokenizer.vocab),
    n_layers=1,
    n_heads=1,
    max_seq_len=MAX_SEQ_LEN,
    n_classes=2
).to(device)
bert

NanoBertForClassification(
  (nano_bert): NanoBERT(
    (embedding): BertEmbeddings(
      (word_embeddings): Embedding(28842, 3)
      (pos_embeddings): Embedding(128, 3)
      (layer_norm): LayerNorm((3,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layers): ModuleList(
        (0): BertLayer(
          (self_attention): BertSelfAttention(
            (heads): ModuleList(
              (0): BertAttentionHead(
                (query): Linear(in_features=3, out_features=3, bias=True)
                (key): Linear(in_features=3, out_features=3, bias=True)
                (values): Linear(in_features=3, out_features=3, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
            )
            (proj): Linear(in_features=3, out_features=3, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (layer_norm1): LayerNorm((3,), eps=1e-05, elementwise

In [42]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [43]:
count_parameters(bert)

87083

In [44]:
optimizer = torch.optim.Adam(bert.parameters(), lr=LEARNING_RATE)

for i in range(NUM_EPOCHS):
    print(f'Epoch: {i + 1}')
    train_loss = 0.0
    train_preds = []
    train_labels = []

    bert.train()
    for step, batch in enumerate(tqdm(dataloader.get_split('train'), total=dataloader.steps('train'))):
        logits = bert(batch['input_ids'].to(device)) # (B, Seq_Len, 2)

        probs = F.softmax(logits[:, 0, :], dim=-1).cpu()
        pred = torch.argmax(probs, dim=-1) # (B)
        train_preds += pred.detach().tolist()
        train_labels += [l.item() for l in batch['label_ids']]

        loss = F.cross_entropy(logits[:, 0, :].cpu(), batch['label_ids'])

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    val_loss = 0.0
    val_preds = []
    val_labels = []

    bert.eval()
    for step, batch in enumerate(tqdm(dataloader.get_split('val'), total=dataloader.steps('val'))):
        logits = bert(batch['input_ids'].to(device))

        probs = F.softmax(logits[:, 0, :], dim=-1).cpu()
        pred = torch.argmax(probs, dim=-1) # (B)
        val_preds += pred.detach().tolist()
        val_labels += [l.item() for l in batch['label_ids']]

        loss = F.cross_entropy(logits[:, 0, :].cpu(), batch['label_ids'])

        val_loss += loss.item()

    print()
    print(f'Train loss: {train_loss / dataloader.steps("train")} | Val loss: {val_loss / dataloader.steps("val")}')
    print(f'Train acc: {accuracy_score(train_labels, train_preds)} | Val acc: {accuracy_score(val_labels, val_preds)}')
    print(f'Train f1: {f1_score(train_labels, train_preds)} | Val f1: {f1_score(val_labels, val_preds)}')

481it [00:02, 187.55it/s]                         
61it [00:00, 235.13it/s]                        



Train loss: 0.6532830839976669 | Val loss: 0.6448634972174963
Train acc: 0.5944224143535071 | Val acc: 0.6233090530697191
Train f1: 0.34758966851406464 | Val f1: 0.4084967320261438


481it [00:02, 212.14it/s]                         
61it [00:00, 168.70it/s]                        



Train loss: 0.5837939226999879 | Val loss: 0.5605499883492787
Train acc: 0.6954430215172593 | Val acc: 0.7148803329864725
Train f1: 0.6167689161554193 | Val f1: 0.6509554140127388


481it [00:02, 179.94it/s]                         
61it [00:00, 229.79it/s]                        


...


Train loss: 0.17004232505375208 | Val loss: 0.090723361990725
Train acc: 0.9410388090749529 | Val acc: 0.9760665972944849
Train f1: 0.9279415269722731 | Val f1: 0.97084917617237


481it [00:02, 223.89it/s]                         
61it [00:00, 227.23it/s]                        



Train loss: 0.18334991042890275 | Val loss: 0.09674443867988884
Train acc: 0.9364233244490672 | Val acc: 0.9745057232049948
Train f1: 0.9223069590085795 | Val f1: 0.9689283449587826


481it [00:02, 221.68it/s]                         
61it [00:00, 222.79it/s]                        



Train loss: 0.16519838607443185 | Val loss: 0.09134297098498792
Train acc: 0.9446141844893714 | Val acc: 0.9781477627471384
Train f1: 0.9323057365326554 | Val f1: 0.9734177215189873
